# Supervised learning for adversarial manipulation detection

In [1]:
# Paths
dataset_root = '/local-scratch/mrivas/dmasmont/misc/lnpfoo/'

In [2]:
# Importing libraries
print('Start importing required libraries...')
import os, sys, time, allel, yaml, math, gzip, torch
sys.path.append('../LAI-Net/')
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
from scipy.interpolate import interp1d
from matplotlib import cm
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import pickle as pkl
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier

from sklearn.metrics import balanced_accuracy_score
print('Done importing, using pytorch version {}...'.format(torch.__version__))

Start importing required libraries...
Done importing, using pytorch version 1.12.1+cu102...


In [3]:

def load_dataset(path, verbose=True):
    with open(path, 'rb') as handle:
        dict_dataset = pkl.load(handle)


    real_seqs = np.array(dict_dataset['real_seqs'])
    if len(real_seqs.shape) == 3:
        real_seqs = real_seqs.reshape(real_seqs.shape[0]*real_seqs.shape[1],real_seqs.shape[2])
    fake_seqs = np.array(dict_dataset['fake_seqs'])
    if len(fake_seqs.shape) == 3:
        fake_seqs = fake_seqs.reshape(fake_seqs.shape[0]*fake_seqs.shape[1],fake_seqs.shape[2])


    if verbose:
        print('Real and fake shapes are ', real_seqs.shape, fake_seqs.shape)
        
    train_x = np.concatenate([real_seqs[0::2,:], fake_seqs[0::2,:]])
    train_y = np.concatenate([np.zeros(real_seqs[0::2,:].shape[0]), np.ones(fake_seqs[0::2,:].shape[0])])

    test_x = np.concatenate([real_seqs[1::2,:], fake_seqs[1::2,:]])
    test_y = np.concatenate([np.zeros(real_seqs[1::2,:].shape[0]), np.ones(fake_seqs[1::2,:].shape[0])])
    
    if verbose:
        print('Training (x,y) and testing (x,y) shapes are ', train_x.shape, train_y.shape, test_x.shape, test_y.shape)
    
    return train_x, train_y, test_x, test_y


In [4]:

dataset_list = ['dataset_manipulated.pkl', 'dataset_Neural-ADMIXTURE.pkl', 'dataset_LAI-Net.pkl']

for dataset_name in dataset_list:
    print('Processing dataset ', dataset_name)
    
    path = dataset_root+dataset_name

    train_x, train_y, test_x, test_y = load_dataset(path)
    
    method_name = ['LogReg', 'KNN', 'MLP', 'XGB']
    method_list = [LogisticRegression(), KNeighborsClassifier(), MLPClassifier(), xgb.XGBClassifier()]


    for method, m_name in zip(method_list, method_name):
        method.fit(train_x, train_y)
        y_pred = method.predict(test_x)
        acc = balanced_accuracy_score(test_y, y_pred)
        print('Accuracy is ', acc, m_name)

Processing dataset  dataset_manipulated.pkl
Real and fake shapes are  (2880, 516800) (50, 516800)
Training (x,y) and testing (x,y) shapes are  (1465, 516800) (1465,) (1465, 516800) (1465,)
Accuracy is  1.0 LogReg
Accuracy is  0.5 KNN
Accuracy is  1.0 MLP


/home/dmasmont/anaconda3/envs/pytorch/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:12:06] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy is  0.8400000000000001 XGB
Processing dataset  dataset_Neural-ADMIXTURE.pkl
Real and fake shapes are  (2880, 516800) (2880, 516800)
Training (x,y) and testing (x,y) shapes are  (2880, 516800) (2880,) (2880, 516800) (2880,)


/home/dmasmont/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy is  0.8868055555555556 LogReg
Accuracy is  0.5118055555555555 KNN
Accuracy is  0.8659722222222221 MLP


...

[17:40:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy is  0.89375 XGB
Processing dataset  dataset_LAI-Net.pkl
Real and fake shapes are  (2880, 516800) (2880, 516800)
Training (x,y) and testing (x,y) shapes are  (2880, 516800) (2880,) (2880, 516800) (2880,)


/home/dmasmont/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy is  0.8659722222222223 LogReg
Accuracy is  0.5048611111111111 KNN
Accuracy is  0.8579861111111111 MLP


/home/dmasmont/anaconda3/envs/pytorch/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:49:18] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy is  0.8659722222222223 XGB
